## __Desafío - Introducción a Big Data__

### Ejercicio 1: Ingesta de datos semiestructurados
- Utilizando las librerías `requests` y `json`, realice una consulta a la [API BallDontLie](https://www.balldontlie.io/#getting-started).
- Mediante esta API, genere una consulta de 100 juegos (busquen el endpoint `Get All Games` en la página, dentro de `Games`) . Puede usar la opción `per_page=100` en la dirección de la API para lograr esto.
- Guarde la consulta en una variable, y asegúrese que ésta sea semiestructurada.
Consulte los datos y metadatos asociados.
- Dentro de la lista de datos, comente cuáles son las llaves de cada registro.


In [9]:
import requests
import pandas as pd
import numpy as np

In [2]:
r = requests.get('https://www.balldontlie.io/api/v1/games?per_page=100')

In [3]:
response = r.json()['data']
print("Ejemplo de Dato")
print(response[0])

Ejemplo de Dato
{'id': 47179, 'date': '2019-01-30T00:00:00.000Z', 'home_team': {'id': 2, 'abbreviation': 'BOS', 'city': 'Boston', 'conference': 'East', 'division': 'Atlantic', 'full_name': 'Boston Celtics', 'name': 'Celtics'}, 'home_team_score': 126, 'period': 4, 'postseason': False, 'season': 2018, 'status': 'Final', 'time': ' ', 'visitor_team': {'id': 4, 'abbreviation': 'CHA', 'city': 'Charlotte', 'conference': 'East', 'division': 'Southeast', 'full_name': 'Charlotte Hornets', 'name': 'Hornets'}, 'visitor_team_score': 94}


In [4]:
meta = r.json()['meta']
print("Ejemplo de Meta")
print(meta)

Ejemplo de Meta
{'total_pages': 512, 'current_page': 1, 'next_page': 2, 'per_page': 100, 'total_count': 51163}


In [5]:
keys = set()
for i in response:
    keys.update(i.keys())
print("Las llaves de cada Dato son: ")
print(keys)

Las llaves de cada Dato son: 
{'time', 'period', 'status', 'season', 'visitor_team_score', 'date', 'visitor_team', 'postseason', 'home_team', 'id', 'home_team_score'}


Ejercicio 2: Organización de los datos
- En base a los datos consultados en la API, genere un objeto `pd.DataFrame` con la
siguiente estructura:
```python
games_df.head(2)
```
- Para obtener la estructura, sólo puede utilizar operaciones map y comprensiones de
lista

In [6]:
games_df = pd.DataFrame([[
    game['season'],
    game['period'],
    game['home_team']['full_name'],
    game['home_team']['conference'],
    game['home_team']['division'],
    game['visitor_team']['full_name'],
    game['visitor_team']['conference'],
    game['visitor_team']['division'],
    game['home_team_score'],
    game['visitor_team_score'],
    int(game['home_team_score']) - int(game['visitor_team_score']),
    int(game['visitor_team_score']) - int(game['home_team_score']),
    ] for game in response], columns=['season', 'period', 'home', 'home_conference', 'home_division', 'visitor', 'visitor_conference', 'visitor_division', 'home_score', 'visitor_score', 'home_differential', 'visitor_differential'])
games_df.head(2)

,season,period,home,home_conference,home_division,visitor,visitor_conference,visitor_division,home_score,visitor_score,home_differential,visitor_differential
0,2018,4,Boston Celtics,East,Atlantic,Charlotte Hornets,East,Southeast,126,94,32,-32
1,2018,4,Boston Celtics,East,Atlantic,LA Clippers,West,Pacific,112,123,-11,11


### Ejercicio 3: El efecto de jugar de local
- Genere una columna en el `pd.DataFrame` que identifique si el equipo que jugó de
local ganó (1) o no (0).
- Repita el procedimiento para identificar si el equipo que jugó de visitante ganó (1) o
no (0).
- Reporte cuáles son los primeros y últimos 5 equipos en cuanto a desempeño por
jugar local.
- Reporte cuáles son los primeros y últimos 5 equipos en cuanto a desempeño por
jugar de visita.


In [12]:
# Agregar las victorias como local y visita
games_df['home_win'] = np.where(games_df['home_differential'] > 0, 1, 0)
games_df['visitor_win'] = np.where(games_df['visitor_differential'] > 0, 1, 0)
# Conseguimos el promedio por equipo de victorias como local y visita
games_df_local = games_df.groupby(['home'])['home_win'].mean().sort_values(ascending=False)
games_df_visita = games_df.groupby(['visitor'])['visitor_win'].mean().sort_values(ascending=False)

In [19]:
# Mostramos los mejores y peores 5 equipos como local
print("Los mejores 5 equipos como local son: ")
print(games_df_local.head(5))
print("Los peores 5 equipos como local son: ")
print(games_df_local.tail(5))


Los mejores 5 equipos como local son: 
home
Miami Heat                1.0
Toronto Raptors           1.0
San Antonio Spurs         1.0
Denver Nuggets            1.0
Portland Trail Blazers    1.0
Name: home_win, dtype: float64
Los peores 5 equipos como local son: 
home
Atlanta Hawks            0.2
Charlotte Hornets        0.0
Phoenix Suns             0.0
New York Knicks          0.0
Oklahoma City Thunder    0.0
Name: home_win, dtype: float64


In [20]:
# Mostramos los mejores y peores 5 equipos como visita
print("Los mejores 5 equipos como visita son: ")
print(games_df_visita.head(5))
print("Los peores 5 equipos como visita son: ")
print(games_df_visita.tail(5))

Los mejores 5 equipos como visita son: 
visitor
Indiana Pacers     1.000000
Milwaukee Bucks    1.000000
LA Clippers        1.000000
Boston Celtics     0.666667
Brooklyn Nets      0.666667
Name: visitor_win, dtype: float64
Los peores 5 equipos como visita son: 
visitor
Minnesota Timberwolves    0.0
New York Knicks           0.0
Phoenix Suns              0.0
Sacramento Kings          0.0
Atlanta Hawks             0.0
Name: visitor_win, dtype: float64


### Ejercicio 4: Obteniendo el porcentaje de ganar local y de visita
- Genere un nuevo objeto que guarde el porcentaje de juegos ganados como local __por equipo__.
- Repita lo mismo para los juegos donde el equipo fue visitante.
- ¿Qué equipos tienen iguales chances de ganar como visitante o local?


In [18]:
# Guardamos la serie como porcentaje
win_as_local_percent = games_df_local * 100
win_as_visita_percent = games_df_visita * 100
# Los juntamos en un DataFrame
games_df_percent = pd.concat([win_as_local_percent, win_as_visita_percent], axis=1)
games_df_percent.columns = ['local_win_percent', 'visita_win_percent']
# Imprimimos aquellos equipos que tienen el mismo porcentaje de victorias tanto como local o visita
print("Los equipos que tienen el mismo porcentaje de victorias como local y visita son: ")
print(games_df_percent[games_df_percent['local_win_percent'] == games_df_percent['visita_win_percent']])

Los equipos que tienen el mismo porcentaje de victorias como local y visita son: 
                      local_win_percent  visita_win_percent
Los Angeles Lakers            50.000000           50.000000
New Orleans Pelicans          33.333333           33.333333
Phoenix Suns                   0.000000            0.000000
New York Knicks                0.000000            0.000000
